# London OpenStreeMap Data Analysis by Jennifer Penuelas 

This notebook documents my process of wrangling, cleaning, and exploring the OpenStreetMap data of London, England, United Kingdom. This is not an area I live in or have travelled to, which presented difficult when thinking about fixing problems outside of field type inconsistencies, spelling inconsistencies, and general user errors. Thus, no information was changed or added that could more accurately represent the area, as I am not familiar enough with it to suggest edits.
    
The data was obtained using Mapzen's metro extracts, specifically searching for London as the extract I was interested in. The search results are separated into two headers "POPULAR EXTRACTS READY FOR DOWNLOAD NOW:" and "TO MAKE A CUSTOM EXTRACT:", I worked with the London data linked first under the "TO MAKE A CUSTOM EXTRACT:" header. [Follow this link to search Mapzen metro extracts.](https://mapzen.com/data/metro-extracts/)

The file size of the London extract I originally asked for is 1.25 GB but for the sake of this project and analysis, I worked on a sample of the London data which was of size 63.24 MB. 

Additionally, I would like to credit some of the ideas behind the outline of this analysis to [Carl Ward's sample project.](https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md)

## London OSM Data Overview

The subsections below, give a general overview of the London data, as per the sample generated. 

In the subsections "Unique Users:", "Tag Counts:", and "Key Types:", I specify different outputs for a sample file that is about 120 MB as well as for the sample file of size 63.24 MB used for this analysis. This is because generating a sample file is quick whereas generating a database to query for things like "Number of Nodes and Ways" takes much longer. The files that give the information in the specified subsections work on the sample file and so they are quick to run. However, generating a database with the larger sample file is ridiculously slow so I had to trash it and use a smaller sample to build the database I queried. 

The time delay occurs because to build the database we first write the data into a schema that can be written into a tabular format which can then be easily inputted into the database. Then before writing the schema into tabular format, the schema is validated and this validation takes too long to do efficiently on a personal computer. However, since I did have the extra information from earlier runs of my code, I decided to include it rather than trash it.


### 1. File sizes:

* london_data.osm - 1.25 GB 
* london_sample.osm - 63.24 MB
> This sample was created by running `python sampling_osm.py` file with k=20, thus getting every 20th top level element. 
* nodes.csv - 21.52 MB
* nodes_tags.csv - 19.11 MB
* ways.csv - 2.59 MB
* ways_tags.csv - 26.52 MB
* ways_nodes.csv - 48.54 MB
> All .csv files are created by running `python data.py`. Note, that this code takes quite some time to run.


### 2. Unique Users:

There are 3238 unique users, get this value by running `python users.py`.

**EXTRA:** Running `users.py` on the larger sample file gives 4018 unique users. 

I would expect that as the sample size gets closer to the original size of the dataset, the difference in unique users would near zero whereas here the difference is 780 users. 

### 3. Number of Nodes and Ways:



There are 262,725 nodes with 60,636 node tags. 

There are 43,836 ways with 342,882 way nodes and 127,288 way tags. 

These were found using a query like this one: `SELECT count(*) FROM nodes;`. This query specifically gives you the number of nodes, to get all other values you would replace 'nodes' in the query with nodes_tags, ways_nodes, or ways_tags.

### 4. Tag counts: 

The following dictionary is returned by the `count_tags` function in the `count_tags.py` file. Run the file by running `python count_tags.py` in the shell from within the folder containing the data files and the code files. This dictionary describes the types of tags we encounter in the data and how many of each there are. 

```
{'member': 14718,
 'nd': 342882,
 'node': 262725,
 'osm': 1,
 'relation': 1062,
 'tag': 193475,
 'way': 43836}
```

**EXTRA:** For the larger sample file, the dictionary returned is: 

```
{'member': 29566,
 'nd': 685606,
 'node': 525449,
 'osm': 1,
 'relation': 2124,
 'tag': 386331,
 'way': 87672}
```

In both cases, you see that the order of which tags you see the least to which you see the most is: osm, relation, member, way, tag, node, and nd. Between the larger and smaller file, the differences in the number of each tag you see are: osm - 0, relation - 1062, member - 14848, way - 43836, tag - 192856, node - 262724, and nd - 342724. In all cases, the number of times you see each tag basically doubles which makes sense since the size of the larger file is double that of the smaller file. However, I did not expect the pattern to be so uniform throughout the tag count.  

### 5. Key types:

Running `python key_types.py` prints out a dictionary of dictionaries which describe potential problems in the data with a count of how many times it encounters the specific probable problem and the k attribute value fitting each problem description for each `<tag>`. Below is a portion of what this dictionary looks like:

```
{'lower': {'count': 146878,
           'text_values': set(['abutters',
                               'access',
                               'accessto',
                               'active_traffic_management',
                               'address',...])},
 'lower_colon': {'count': 38287,
                 'text_values': set(['abandoned:aeroway',
                                     'abandoned:amenity',
                                     'access:conditional',
                                     'access:note',
                                     'addr:apartment',...])},
 'other': {'count': 8308,
           'text_values': set(['BROADHAB',
                               'CREATEDATE',
                               'CREATEDBY',
                               'FIXME',
                               'FIXME:nsl',...])},
 'problemchars': {'count': 2,
                  'text_values': set(['hgv:7.5', 'maxweight:7.5'])}}
```

**Extra:** The larger file has the following dictionary:

```
{'lower': {'count': 293473,
           'text_values': set(['abutters',
                               'access',
                               'accessto',
                               'active_traffic_management',
                               'address',...])},
 'lower_colon': {'count': 76097,
                 'text_values': set(['abandoned:aeroway',
                                     'abandoned:amenity',
                                     'abandoned:highway',
                                     'abandoned:sport',
                                     'access:conditional',...])},
 'other': {'count': 16759,
           'text_values': set(['BROADHAB',
                               'CREATEDATE',
                               'CREATEDBY',
                               'ENSISID',
                               'FIXME',...])},
 'problemchars': {'count': 2,
                  'text_values': set(['hgv:7.5', 'maxweight:7.5'])}}
```

Some things to notice:

Although the larger file is twice the size, there are not twice the amount of possible 'problemchars'. That is a good thing! In my opinion it means that this data is well kept or contributed to and fixed often, especially since it is an area of popular interest. Additionally, notice that the values are potentially problematic because of the period in it so there is consistency within even the 'problemchars' thus allowing for more leniency when cleaning.

Also, the text_values set in both of the 'lower_colon' and 'other' dictionaries from both files contain different values. Recall that when a sample is created, you decide the size by setting k equal to some value. The larger k is the smaller the sample, since k specifies which top level elements you want. In the smaller file, we set k = 20 thus getting every 20th element whereas the larger file was obtained by setting k = 10. Thus, the larger file contains the values from the smaller file as well where the smaller file contains every other value of that in the large file.   

### 6. Additional Interesting Overview Facts:

In this section there are a few interesting overview facts, based on my own personal interests.

#### Cafes!

Using the queries: 
```
    SELECT value, count(*) as search_amount 
    FROM nodes_tags
    WHERE value = 'cafe';
    
    SELECT value, count(*) as search_amount 
    FROM ways_tags
    WHERE value = 'cafe';
    ```
We find that there are 205 cafes, as the first query returns [(u'cafe', 128)] and the second returns [(u'cafe', 77)].

#### Abandoned Places! 

The query below tells us that there are some abandoned places in our data, specifically the ones we see in our data are railways.

Query: 
```
    SELECT *
    FROM ways_tags
    WHERE value = 'abandoned';
    ```
Output:
```
    [(165810010, u'railway', u'abandoned', u'regular'),
     (22617717, u'railway', u'abandoned', u'regular'),
     (184920456, u'railway', u'abandoned', u'regular'),
     (17548926, u'railway', u'abandoned', u'regular'),
     (139309617, u'railway', u'abandoned', u'regular'),
     (36854480, u'railway', u'abandoned', u'regular')]
    ```

#### User Contributions!

The query `SELECT user, count(*) AS contributions FROM nodes GROUP BY user ORDER BY contributions DESC LIMIT 10;` tells us that the top ten users across the node elements are: 
```
[(u'busdoc', 15378),
 (u'Rondon237', 8823),
 (u'SDavies', 8396),
 (u'Derick Rethans', 8286),
 (u'Tom Chance', 8136),
 (u'Paul The Archivist', 6998),
 (u'Amaroussi', 6474),
 (u'TimSC_Data_CC0_To_Andy_Allan', 6343),
 (u'Steve Chilton', 5820),
 (u'ecatmur', 5731)]
 ```
Furthermore, the same query but from the ways table tells us that the top ten users contributing to way elements are:
```
[(u'busdoc', 2101),
 (u'SDavies', 1896),
 (u'Welshie', 1777),
 (u'Derick Rethans', 1755),
 (u'Amaroussi', 1601),
 (u'Paul The Archivist', 1425),
 (u'Tom Chance', 1209),
 (u'Rondon237', 1116),
 (u'harg', 959),
 (u'peregrination', 868)]
```
I find it interesting that the eight, ninth, and tenth contributors to node elements are not in the top ten for way elements. Also, that the third highest way contributor, 'Welshie', is not a a part of the top top ten for nodes. Now, recall from section three that there are significantly more nodes that ways, so the fact that some top node contributors are not in the top way contributors is expected. However, the the opposite is more surprising. 

## Making Improvements to the Data

### The Problems:

1) The 'user' and 'uid' values and columns created a problem both when generating the dictionaries to fit the schema to write the data into csv files as well as in querying.
> a) When generating the csv files and inserting the data into the database, there were nodes which did not have 'user' and 'uid' attributes, creating a problem when trying to insert their values into the dictionaries.

> b) When querying we saw user values like 'z\u2006x\u2006f\u2006m' and '\u042e\u043a\u0430\u0442\u0430\u043d'. These types of user names are obviously not conciously created by a human user.  

2) Node and way tags of type 'name' are problematic. They generally do not offer much understandable information, as such it is difficult to make changes that would make the information in these tags clearer. 
> a) The following query `SELECT key, value FROM nodes_tags WHERE type = 'name';` on both the nodes_tags table and the ways_tags tables give the following results:
```
[...
  (u'ru', u'\u0425\u043e\u043b\u043b\u043e\u0443\u044d\u0439-\u0440\u043e\u0443\u0434'),
  (u'eo', u'Londono'),
  (u'cv', u'\u041b\u043e\u043d\u0434\u043e\u043d'),
  (u'cdo', u'L\xf9ng-d\u016dng'),
  (u'es', u'Londres'),
  ...]```
 
> and       
```
[...
  (u'ru', u'\u043c\u043e\u0441\u0442 \u0412\u0430\u0442\u0435\u0440\u043b\u043e\u043e'),
  (u'cy', u'Abaty San Steffan'),
  (u'be', 
   u'\u041f\u0430\u0440\u043b\u0430\u043c\u0435\u043d\u0442\u0441\u043a\u0430\u044f \u043f\u043b.'),
  (u'en', u"Bagley's Lane"),
  (u'fr', u'Abbaye de Westminster'),
  ...]
  ``` 

3) Finally this data is organized to the point where there are some elements in which the k values signify a possible fix. The following query `SELECT key, value FROM nodes_tags WHERE type = 'fixme';` on both of the nodes_tags table and the ways_tags table with also with the condition `WHERE key = 'FIXME'` gives the areas that should be looked at for fixing and also a description of why or how they should be fixed.
> a) The query `SELECT key, value FROM ways_tags WHERE type = 'fixme';` gives:     
```
[...
 (u'FIXME', u'Check this.'),
 (u'FIXME', u'yahoo shows a large island in the road?'),
 (u'FIXME', u'?identify?'),
 (u'FIXME', u'name? blurry photo'),
 (u'FIXME',
  u'There seem to be too many restaurants here - some may have closed or been put in the wrong place.'),
  ...]
  ``` 
> and `SELECT key, value FROM ways_tags WHERE key = 'FIXME';` gives:
```
[...
(u'FIXME', u'Trace from Yahoo Image.'),
 (u'FIXME', u'OS Locator says "Hadden Road"'),
 (u'FIXME', u'Trace from Yahoo Image.'),
 (u'FIXME', u'wheelchair tagging?'),
 (u'FIXME', u'double check this street name (bad photo)'),
...]
  ``` 

### The Fixes:

1a) The simplest fix was to check if the node element had the attributes 'user' and 'uid'; if not, then the value of these would default to 'NO_USER' and 0, respectively. In the end, we find that there are only six nodes that required this alteration. The query `SELECT * FROM nodes WHERE user = 'NO_USER';` gives the following result: 
```
[(17805033, 51.360339, -0.3631331, u'NO_USER', 0, 1, 128532, u'2006-10-07T20:08:04Z'),
 (17806151, 51.3622816, -0.3572716, u'NO_USER', 0, 1, 128532, u'2006-10-07T20:18:18Z'),
 (17806863, 51.3550082, -0.3375747, u'NO_USER', 0, 1, 128532, u'2006-10-07T20:30:28Z'),
 (17807424, 51.3654952, -0.355275, u'NO_USER', 0, 1, 128532, u'2006-10-07T20:44:26Z'),
 (17807459, 51.3650557, -0.3493271, u'NO_USER', 0, 1, 128532, u'2006-10-07T20:44:28Z'),
 (17916421, 51.4900279, -0.2815898, u'NO_USER', 0, 1, 129695, u'2006-10-08T23:18:04Z')]
 ```
 **NOTE:** The order of the columns is (id, lat, lon, user, uid, version, changeset, timestamp)
 
Interestingly, these occurences have more than their made up user name and user id values in common. The version and changeset values are all the same and the timestamp values are all within minutes of each other. This leads one to question if these nodes are created when initiating the OpenStreetMap data for this particular region. I believe that this distintion, if it is true could be a valuable improvement for the data, then the 'user' value can go from the more ambiguous 'NO_USER' to 'SETUP'.

> **Benefits:** Implementing this fix would make the data more clear upon first read through and would indicate some overall timeline within the data itself, particularly with the data initiation.

> **Anticipated Problems:** This is more time consuming when creating the data.

1b) The query `SELECT user, count(*) AS contributions FROM nodes GROUP BY user ORDER BY contributions DESC;` allowed for the discovery of the suspicious user names referenced above. I did not fix them as these values were not disruptive in my own queries. However, if this would be a problem for anyone using this data, I suggest ruling out these potentially problematic username values by searching for them using regular expressions which look for '\' or any other problematic characters in the node's user attribute value and then replacing those that are problematic to you with another easily recognizable value. Additionally, from a paranoid user, I would question the authenticity of the data from these users so maybe ignoring these rows completely could be a viable option and if these did occur from suspicious use of the OSM database can OSM catch it?    

> **Benefits:** This fix makes the data clearer to read.

> **Anticipated Problems:** This fix assumes a lot about problematic strings and the easiest fix, to replace them with something more noticeable, does create blindness to the problem in the analysis of your data. Additionally, creating some software that monitors suspicious activity or inputs to the OSM database is time consuming and maybe not worth it for this type of data contributions as their main information comes from users adding to it. 

2a) In my opinion, the data found through the queries above raise too many questions to be good data. Some of these questions are: What do the keys mean? Are the keys abbreviations or codes? If so, can I get access to them so that I can alter the data I work with and make it more informative at first glance. Then, again we see problematic strings for the value columns which contain '\' characters so that the values are useless. What could these string be referring to? Finally, since these are tags of type 'name' what do the names refer to? To fix and maybe answer some of these questions, if the keys for name tags are codes, provide a way for decoding, even if it is to be done manually, with the original download extract and again, dealing with problematic string would be done in the way suggested in point 1b.

> **Benefits:** This fix gives more documentation on the OSM data and how child tags are defined and it would make cleaning for people using the data much easier.

> **Anticipated Problems:** This fix requires someone to document the data and how name type tags are defined, which is time consuming and perhaps not ideal especially since this is regional data which could change from area to area. 

3a) Note that in the final query of this section the results mention Yahoo, which gives some indication that outside resources are somehow used to construct this data and could then be used to fix it without a human needing to conciously and manually fix the issues. Since there are latitude and longitudes associate with each element usually, the data could somehow use outside map applications and apps like Yelp to address some issues like for example whether a restaurant that used to be there is still open. It can see how much people still go to that location and make assumptions based on this information to deduce the answers they want. 


> **Benefits:** This fix would hopefully solve some problems on its own and would use the power of other applications to create a more clear and accurate representation of the regional data.

> **Anticipated Problems:** This fix would require a lot of permission from other apps to use their data and then would require a lot of work to build something smart enough to go through it and succesfully implement it into the OSM data. 

## Conclusion

Through, this analysis I generally found that the London data is well kept. There is consistency across the same attribute values and also consistency in things that may seem problematic. This was exemplified in both the 'Key Types' section where the dictionaries from both the smaller and larger file contain the same number and the same values for potentially problematic k values. Additionally, these 'key_type' dictionaries shed light how how well kept this London data is as there are tags with k values equal to 'FIXME' or 'fixme:date'. We can further see more values of these fix me tags by querying over the nodes_tags and ways_tags tables searching for type equal to fix me. I did audit a particular fix which pertained to date in the `audit.py` file but additionally I discussed this further in the 'Making Improvement's to the Data' section where we discussed the possibility of making other applications such as a maps app or Yelp, etc, fix our data for us. Finally, we also see the same consistency in the user columns of the nodes and ways tables and also the value columns of nodes_tags and ways_tags tables when the type is name where we see that potentially problematic user names, at the very least, contain the character '\'.

Some of my favorite discoveries are those made with the idea of being a potential tourist in mind. Particularly, my favorite discovery was made in noticing that the k attribute of elements could have values that included the term 'abandoned'. I then went on to search which locations were deemed abandoned. In my sample, I found only abandoned railways and I would be curious to see if a larger sample would reveal different abandoned places. 